In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.200232,-0.888892,1.524665,-0.506326,0.150994,2.090686,1.145647,1.40439,-1.409437,-2.024626
1,2.202776,0.607151,-0.711372,-1.522102,-0.259607,-2.307091,-1.1653,-1.870384,0.551632,-1.449511
1,-1.215223,0.63889,0.512399,1.136189,-1.202908,1.487095,1.307504,2.19001,0.35992,-1.782716
0,0.063379,0.545179,0.941311,1.653572,-2.221434,1.423115,0.581753,1.07439,2.010467,-3.5149
1,-5.28972,-0.669352,3.301669,4.930371,1.023188,6.495264,-0.939379,1.147088,3.183671,-2.34192


## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
scaled = t.scale(df, cols=features, strategy="standard")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.199798,-0.508845,0.943883,-0.570708,0.41983,0.769539,0.522875,0.562408,-0.73395,-1.254273
1,1.233661,0.350597,-0.524313,-1.146243,0.153935,-1.186411,-1.097783,-1.47176,0.161969,-0.897622
1,-0.530966,0.36883,0.279223,0.359933,-0.456925,0.501086,0.636384,1.050407,0.074385,-1.104255
0,0.129144,0.314995,0.560849,0.65308,-1.1165,0.472631,0.127418,0.357425,0.828441,-2.178449
1,-2.634527,-0.382724,2.110675,2.5097,0.984643,2.728514,-0.939346,0.402582,1.364422,-1.45104


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,target
0,0.199799,-0.508847,0.943888,-0.570711,0.419832,0.769543,0.522878,0.562411,-0.733953,-1.254279,0
1,1.233667,0.350599,-0.524315,-1.146248,0.153935,-1.186417,-1.097789,-1.471767,0.161970,-0.897627,1
2,-0.530968,0.368832,0.279224,0.359935,-0.456928,0.501089,0.636388,1.050412,0.074385,-1.104261,1
3,0.129145,0.314997,0.560852,0.653084,-1.116505,0.472633,0.127419,0.357427,0.828445,-2.178460,0
4,-2.634540,-0.382726,2.110686,2.509712,0.984648,2.728527,-0.939351,0.402584,1.364429,-1.451047,1


In [9]:
%%timeit
scaled = t.scale(df, cols=features, strategy="standard")

2.61 ms ± 42.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


15.9 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
import dsds.transform as t
t.impute(df, cols=features, strategy="median").head(3) 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.200232,-0.888892,1.524665,-0.506326,0.150994,2.090686,1.145647,1.40439,-1.409437,-2.024626
1,2.202776,0.607151,-0.711372,-1.522102,-0.259607,-2.307091,-1.1653,-1.870384,0.551632,-1.449511
1,-1.215223,0.63889,0.512399,1.136189,-1.202908,1.487095,1.307504,2.19001,0.35992,-1.782716


In [12]:
from sklearn.impute import SimpleImputer 
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head(3)

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0.0,0.200232,-0.888892,1.524665,-0.506326,0.150994,2.090686,1.145647,1.404390,-1.409437,-2.024626
1,1.0,2.202776,0.607151,-0.711372,-1.522102,-0.259607,-2.307091,-1.165300,-1.870384,0.551632,-1.449511
2,1.0,-1.215223,0.638890,0.512399,1.136189,-1.202908,1.487095,1.307504,2.190010,0.359920,-1.782716


In [13]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)


125 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
t.impute(df, cols=features, strategy="median")

1.19 ms ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Fscore

In [15]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [16]:
fs._f_score(df, target=target, num_list = features)

array([2.88060581e+03, 8.97413770e-03, 1.69476228e+02, 8.68852207e+03,
       1.16376096e+04, 7.69397014e+02, 4.93347606e+03, 1.03827771e+04,
       1.76492943e+04, 3.72855765e-01])

In [17]:
# The more core, the bigger the difference. Data here is not big enough to show the difference
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.01s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",2880.605809,0.0
"""column_1""",0.008974,0.924528
"""column_2""",169.476228,1.0353e-38
"""column_3""",8688.522069,0.0
"""column_4""",11637.609615,0.0
"""column_5""",769.397014,1.0641e-168
"""column_6""",4933.476058,0.0
"""column_7""",10382.777147,0.0
"""column_8""",17649.294275,0.0


In [18]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


,feature,f_value,p_value
0,column_0,2880.605809,0.000000e+00
1,column_1,0.008974,9.245279e-01
2,column_2,169.476228,1.035336e-38
3,column_3,8688.522069,0.000000e+00
4,column_4,11637.609615,0.000000e+00
5,column_5,769.397014,1.064110e-168
6,column_6,4933.476058,0.000000e+00
7,column_7,10382.777147,0.000000e+00
8,column_8,17649.294275,0.000000e+00
9,column_9,0.372856,5.414531e-01


In [19]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.01s in computing Fscore.


,feature,f_value,p_value
0,column_0,2880.605809,0.000000e+00
1,column_1,0.008974,9.245279e-01
2,column_2,169.476228,1.035336e-38
3,column_3,8688.522069,0.000000e+00
4,column_4,11637.609615,0.000000e+00
5,column_5,769.397014,1.064110e-168
6,column_6,4933.476058,0.000000e+00
7,column_7,10382.777147,0.000000e+00
8,column_8,17649.294275,0.000000e+00
9,column_9,0.372856,5.414531e-01


# Mutual Information Score

In [20]:
# Vastly faster than sklearn. Finished in 0.7s in this run
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 10/10 [00:00<00:00, 13.10it/s]


feature,estimated_mi
str,f64
"""column_8""",0.090184
"""column_7""",0.064032
"""column_4""",0.059749
"""column_0""",0.057873
"""column_3""",0.040985
"""column_6""",0.033473
"""column_2""",0.030144
"""column_5""",0.0194
"""column_9""",0.01073


In [21]:
# A wrapper for more apples to apples comparison
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [22]:
# The reason sklearn's impl is slow is that it did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality. Finished in 4.4s in this run
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_8""",0.090184
"""column_7""",0.064029
"""column_4""",0.05975
"""column_0""",0.057873
"""column_3""",0.040985
"""column_6""",0.033473
"""column_2""",0.030144
"""column_5""",0.019398
"""column_9""",0.01073


# MRMR Feature selection Strategy

In [23]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [24]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [25]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 10/10 [00:06<00:00,  1.63it/s]

Spent 8.98s to compute mrmr.


['column_8',
 'column_4',
 'column_7',
 'column_3',
 'column_6',
 'column_0',
 'column_5',
 'column_2',
 'column_9',
 'column_1']

In [26]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 1999.76it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_8',
 'column_4',
 'column_7',
 'column_3',
 'column_6',
 'column_0',
 'column_5',
 'column_2',
 'column_9',
 'column_1']

In [27]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

INFO:dsds.fs:Running fscore to determine feature relevance...


Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 555.41it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.03s in computing.


['column_8',
 'column_4',
 'column_7',
 'column_3',
 'column_6',
 'column_0',
 'column_5',
 'column_2',
 'column_9',
 'column_1']

# Power Transform

In [28]:
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Spent 0.24s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.203884,-0.937633,1.642856,-0.502813,0.149992,2.128988,1.177659,1.514831,-1.261529,-1.992616
1,2.520246,0.584287,-0.683235,-1.496533,-0.262508,-2.262912,-1.133432,-1.707013,0.58168,-1.43145
1,-1.11589,0.613791,0.528427,1.151781,-1.253924,1.508405,1.348006,2.430603,0.37322,-1.756923
0,0.063758,0.526424,0.990961,1.683729,-2.370455,1.442848,0.591017,1.142623,2.33023,-3.435586
1,-4.232702,-0.698183,3.746853,5.113499,0.986767,6.733468,-0.917648,1.223906,3.890545,-2.301063


In [29]:
from sklearn.preprocessing import power_transform

In [30]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 0.70s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0,0.203884,-0.937633,1.642856,-0.502813,0.149992,2.128988,1.177659,1.514831,-1.261529,-1.992616
1,1,2.520246,0.584287,-0.683235,-1.496533,-0.262508,-2.262912,-1.133432,-1.707013,0.581680,-1.431450
2,1,-1.115890,0.613791,0.528427,1.151781,-1.253924,1.508405,1.348006,2.430603,0.373220,-1.756923
3,0,0.063758,0.526424,0.990961,1.683729,-2.370455,1.442848,0.591017,1.142623,2.330230,-3.435586
4,1,-4.232702,-0.698183,3.746853,5.113499,0.986767,6.733468,-0.917648,1.223906,3.890545,-2.301063


# GC

In [31]:
import gc 
gc.collect()

225